# Code Interpreter Agent

### Connect to the Foundry

In [ ]:
# Import packages
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv(override=True) # Load environment variables from .env file

# Use the connection string to connect to your Foundry project
project_connection_string = os.getenv("AIPROJECT_CONNECTION_STRING")
try:
    project = AIProjectClient.from_connection_string(
        conn_str=project_connection_string, credential=DefaultAzureCredential()
    )
except Exception as e:
    print(f"Failed to connect to project: {e}")
    raise


### Upload the files

In [ ]:
uploaded_file_ids = []

try:
    for local_file in os.listdir("data"):

        # Define the local file path
        local_file_path = os.path.join("data", local_file)

        print(f"Processing File '{local_file_path}'...")

        # Use the upload and poll SDK helper to upload the local file, add them to the vector store,
        # and poll the status of the file batch for completion.
        file = project.agents.upload_file_and_poll(file_path=local_file_path, purpose="assistants")
        uploaded_file_ids.append(file.id)

        print(f"Uploaded file, file ID: {file.id}")

            
    print(f"File IDs: {uploaded_file_ids}")

except FileNotFoundError:
    print(f"Error: The file '{curr_file}' was not found.")
    raise

except Exception as e:
    print(f"An error occurred while processing the Excel file: {e}")
    raise

In [ ]:
uploaded_file_ids = uploaded_file_ids[:2]

### Create the Agent with CodeInterpreter Tool

In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Create file search tool with resources followed by creating agent
code_tool = CodeInterpreterTool(file_ids=uploaded_file_ids)
print(f"CodeInterpreter tool created.")


## need to login using the command azd auth login
agent = project.agents.create_agent(
        model=os.getenv("CHAT_MODEL"),
        name="CodeInterpreterAgent",
        instructions="You are helpful AI assistant.",
     
        tools=code_tool.definitions,
        tool_resources=code_tool.resources
    )
print(f"Created agent, agent ID: {agent.id}")

In [ ]:
# Create a thread
thread = project.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

### Util method message handeling

In [ ]:
def sendMessage(msg):
    # Get messages from the thread
    message = project.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=msg
        )
    print(f"Created message, ID: {message.id}")

    run = project.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
            print(f"Run failed: {run.last_error}")
            raise Exception(f"Run failed: {run.last_error}")

    messages = project.agents.list_messages(thread_id=thread.id, order="asc", after=message.id)
    for msg in messages.data:
        role = msg.role
        content = msg.content[0].text.value
        print(f"{role.capitalize()}: {content}")


In [ ]:
sendMessage("Each file contains a form. Extract the name of the person from each form. Output the result in a table with two columns: file name and name of the person. The file names should be the same as the original file names.")

In [ ]:
sendMessage("Also include the number of stocks owned by each person.")